In [2]:
from ndmg.register import register as ndr
atlas = '/home/eric/atlases/atlas/MNI152_T1_1mm.nii.gz'
t1w = '/data/raw/sub-0025864/ses-1/anat/sub-0025864_ses-1_T1w.nii.gz'
xfm = 't1w2atlas.xfm'
t1w_realigned = 'sub-0025864_ses-1_T1w_space-MNI152.nii.gz'

In [4]:
ndr.register().align(t1w, atlas, xfm)
ndr.register().applyxfm(t1w, atlas, xfm, t1w_realigned)

Executing: flirt -in /data/raw/sub-0025864/ses-1/anat/sub-0025864_ses-1_T1w.nii.gz -ref /home/eric/atlases/atlas/MNI152_T1_1mm.nii.gz -omat t1w2atlas.xfm -dof 12 -bins 256 -cost mutualinfo -searchrx -180 180 -searchry -180 180 -searchrz -180 180
Executing: flirt -in /data/raw/sub-0025864/ses-1/anat/sub-0025864_ses-1_T1w.nii.gz -ref /home/eric/atlases/atlas/MNI152_T1_1mm.nii.gz -out sub-0025864_ses-1_T1w_space-MNI152.nii.gz -init t1w2atlas.xfm -interp trilinear -applyxfm


In [8]:
from ndmg.utils import nuis_utils as ndn
from ndmg.utils import utils as ndu

In [9]:
basename = 'sub-0025864_ses-1_seg'
ndu.extract_brain(t1w_realigned, t1w_realigned, '-f  0.3')
ndn.segment_t1w(t1w_realigned, basename, '-P')

Executing: bet sub-0025864_ses-1_T1w_space-MNI152.nii.gz sub-0025864_ses-1_T1w_space-MNI152.nii.gz -f  0.3
Segmenting Anatomical Image into WM, GM, and CSF...
Executing: fast -t 1 -P -n 3 -o sub-0025864_ses-1_seg sub-0025864_ses-1_T1w_space-MNI152.nii.gz


{'csf_prob': 'sub-0025864_ses-1_seg_pve_0.nii.gz',
 'gm_prob': 'sub-0025864_ses-1_seg_pve_1.nii.gz',
 'wm_prob': 'sub-0025864_ses-1_seg_pve_2.nii.gz'}

In [10]:
import nibabel as nb
seg_t1w = '{}_{}.nii.gz'.format(basename,  'seg')
t1w_im = nb.load(seg_t1w)

In [14]:
t1w_segdat = t1w_im.get_data()
from ndmg.graph.zindex import XYZMorton

In [20]:
dims = t1w_segdat.shape
tclass = {}
for x in range(0, dims[0]):
    for y in range(0, dims[1]):
        for z in range(0, dims[2]):
            if t1w_segdat[x, y, z] > 0:
                tclass[XYZMorton((int(x), int(y), int(z)))] = t1w_segdat[x, y, z]

In [21]:
outf = '../../data/class/class.txt'
with open(outf, 'w')  as f:
    for key, val in tclass.iteritems():
        f.write('{} {}\n'.format(key, val))